In [35]:
import numpy as np 
import pandas as pd 
import re
import nltk
from nltk import FreqDist
from nltk.tokenize import TweetTokenizer
import matplotlib.pyplot as plt
%matplotlib inline
from dataScript import getDataset
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import svm
from sklearn.metrics import accuracy_score

data = getDataset()
#data = data.sample(frac=1).reset_index(drop=True)
data.head()


features = data.iloc[:,0].values
labels = data.iloc[:,1].values
tknzr = TweetTokenizer()
lenth = 0
for sen in features:
    words = tknzr.tokenize(sen)
    if len(words) > lenth:
        lenth = len(words)

print(lenth)

f = open("stop_words.txt", "r", encoding='utf8')
stop = []
for x in f:
  stop.append(x.replace('\n',''))
f.close()
processed_features = []
ss = []

# for i in range(0, len(stop)):
#     s = str(stop[i])
#     s = s.lower()
#     s= re.sub(r'ü', 'u', s)
#     s= re.sub(r'ö', 'o', s)
#     s= re.sub(r'ı', 'i', s)
#     s= re.sub(r'ğ', 'g', s)
#     s= re.sub(r'ç', 'c', s)
#     ss.append(s)

processed_features = []

for sentence in range(0, len(features)):
    processed_feature = re.sub(r'\s\'\s', '\'', str(features[sentence]))
    # processed_feature= re.sub(r'[.!?,\"\[\]]', '', processed_feature)
    
    #processed_feature= re.sub(r'(?<=\d)(\'\s)', '\'', processed_feature)    
    processed_feature = processed_feature.lower()
    # processed_feature= re.sub(r'ü', 'u', processed_feature)
    # processed_feature= re.sub(r'ö', 'o', processed_feature)
    # processed_feature= re.sub(r'ı', 'i', processed_feature)
    # processed_feature= re.sub(r'ğ', 'g', processed_feature)
    # #processed_feature= re.sub(r'ş', 's', processed_feature)
    # processed_feature= re.sub(r'ç', 'c', processed_feature)
    # # Replaces #hashtag with hashtag
    # processed_feature = re.sub(r'#(\S+)', r' \1 ', processed_feature)
    # # Smile -- :), : ), :-), (:, ( :, (-:, :')
    # processed_feature = re.sub(r'(:\s?\)|:-\)|\(\s?:|\(-:|:\'\))', ' EMO_POS ', processed_feature)
    # # Laugh -- :D, : D, :-D, xD, x-D, XD, X-D
    # processed_feature = re.sub(r'(:\s?D|:-D|x-?D|X-?D)', ' EMO_POS ', processed_feature)
    # # Love -- <3, :*
    # processed_feature = re.sub(r'(<3|:\*)', ' EMO_POS ', processed_feature)
    # # Wink -- ;-), ;), ;-D, ;D, (;,  (-;
    # processed_feature = re.sub(r'(;-?\)|;-?D|\(-?;)', ' EMO_POS ', processed_feature)
    # # Sad -- :-(, : (, :(, ):, )-:
    # processed_feature = re.sub(r'(:\s?\(|:-\(|\)\s?:|\)-:)', ' EMO_NEG ', processed_feature)
    # # Cry -- :,(, :'(, :"(
    # processed_feature = re.sub(r'(:,\(|:\'\(|:"\()', ' EMO_NEG ', processed_feature)

    # # Convert more than 2 letter repetitions to 2 letter
    # processed_feature = re.sub(r'(.)\1+', r'\1\1', processed_feature)
    processed_features.append(processed_feature)


#print(processed_features[10:])
# all_words = []
# tknzr = TweetTokenizer()
# for sen in processed_features:
#     words = tknzr.tokenize(sen)    
#     #print(words)
#     filtered_words = [w for w in words if not w in stop]
#     all_words.append(str(filtered_words))
#     #print(filtered_words)


from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

X_train, X_test, y_train, y_test = train_test_split(processed_features, labels, test_size=0.2, random_state=1)
tokenizer = Tokenizer(num_words=2500)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1

maxlen = 35

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)


# define model
model = Sequential()
model.add(Embedding(vocab_size, 64, input_length=maxlen))
model.add(Flatten())
model.add(Dense(250, activation='tanh'))
model.add(Dense(3, activation='sigmoid'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), 
          epochs=10, batch_size=128, verbose=1)

# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

#print(accuracy_score(y_test, predicted_sentiment))

35
Model: "sequential_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_33 (Embedding)     (None, 35, 64)            677376    
_________________________________________________________________
flatten_33 (Flatten)         (None, 2240)              0         
_________________________________________________________________
dense_66 (Dense)             (None, 250)               560250    
_________________________________________________________________
dense_67 (Dense)             (None, 3)                 753       
Total params: 1,238,379
Trainable params: 1,238,379
Non-trainable params: 0
_________________________________________________________________
Train on 2399 samples, validate on 600 samples
Epoch 1/10
2399/2399 [==============================] - 1s 474us/step - loss: 1.0795 - accuracy: 0.4231 - val_loss: 1.0704 - val_accuracy: 0.4500
Epoch 2/10
2399/2399 [=======================